In [1]:
import pandas as pd
import numpy as np
import math
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import RandomForestRegressor



pd.set_option("display.max_rows", None, "display.max_columns", None)


sales=pd.read_csv("Dataset_3_sales.csv", sep="|",header=None)
sales.columns = sales.iloc[0, :].tolist()
sales = sales.drop(0, axis = 0).reset_index(drop = True)

ID = sales.loc[:,'index']
sales = sales.drop('index',axis = 1)
sales = sales.rename(columns={sales.columns[0]: "id"})
sales = sales.drop('id',axis =  1) 
sales = sales.drop(sales.columns[[0,1,2,3,4,5,6,7]],axis =  1) 


#print(sales)

In [2]:
sales['Item_Identifier'] = sales['Item_Identifier'].astype(str)
sales.loc[sales['Item_Identifier']=='-400','Item_Identifier']=np.nan
sales.loc[sales['Item_Identifier']=='Allez au boulot ! :)','Item_Identifier']=np.nan
sales.loc[sales['Item_Identifier']=='nan','Item_Identifier']=np.nan
ID_Item = sales.loc[:,'Item_Identifier']
sales=sales.drop('Item_Identifier', axis=1)

In [3]:
sales['Item_Weight'] = pd.to_numeric(sales['Item_Weight'], errors='coerce')
for x, y in zip(sales.Item_Weight, sales.index): 
        rep = math.isnan(sales.Item_Weight[y]) #rep récupère l'emplacement des NaN dans product_cost
        if rep == True : #si rep est un NaN 
            sales['Item_Weight'][y] = sales['Item_Weight'].mean() #remplace la valeur de product_cost par la moyenne
#print(sales.Item_Weight)
#sales.Item_Weight.describe()

<ipython-input-3-50fdd0918773>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['Item_Weight'][y] = sales['Item_Weight'].mean() #remplace la valeur de product_cost par la moyenne


In [4]:
sales['Item_Fat_Content'] = sales['Item_Fat_Content'].astype(str)
sales['Item_Fat_Content'] = sales['Item_Fat_Content'].replace(['low fat', 'LF'], 'Low Fat')
sales['Item_Fat_Content'] = sales['Item_Fat_Content'].replace(['reg'], 'Regular')
sales['Item_Fat_Content'] = sales['Item_Fat_Content'].replace(['nan'], 'None')

#print(sales.Item_Fat_Content)

In [5]:
sales['Item_Visibility'] = sales['Item_Visibility'].astype(float)
#print(sales.Item_Visibility)


In [6]:
sales['Item_Type'] = sales['Item_Type'].astype(str)
#print(sales.Item_Type)
#sales.groupby('Item_Type').sum()

In [7]:
sales['Item_MRP'] = sales['Item_MRP'].astype(float)
#print(sales.Item_MRP)

In [8]:
sales['Outlet_Identifier'] = sales['Outlet_Identifier'].astype(str)
sales.loc[sales['Outlet_Identifier']=='nan','Outlet_Identifier']=np.nan
ID_Outlet= sales.loc[:,'Outlet_Identifier']
sales=sales.drop('Outlet_Identifier', axis=1)

In [9]:
sales['Outlet_Establishment_Year'] = sales['Outlet_Establishment_Year'].astype(float)
#print(sales.Outlet_Establishment_Year)

In [10]:
sales['Outlet_Size'] = sales['Outlet_Size'].astype(str)
sales.loc[sales['Outlet_Size']=='nan','Outlet_Size']="0"
#sales.groupby('Outlet_Size').sum()
#print(sales.Outlet_Size)

In [11]:
sales['Outlet_Location_Type'] = sales['Outlet_Location_Type'].astype(str)
sales.loc[sales['Outlet_Location_Type']=='nan','Outlet_Location_Type']=np.nan
#sales.groupby('Outlet_Location_Type').sum()
#print(sales.Outlet_Location_Type)

In [12]:
sales['Outlet_Location_Type'] = sales['Outlet_Location_Type'].astype(str)
sales.loc[sales['Outlet_Location_Type']=='nan','Outlet_Location_Type']=np.nan
#sales.groupby('Outlet_Type').sum()
#print(sales.Outlet_Type)

In [13]:
sales['Item_Outlet_Sales'] = sales['Item_Outlet_Sales'].astype(float)
#print(sales.Item_Outlet_Sales)

In [14]:
sales_OHE=pd.DataFrame(OneHotEncoder().fit_transform(sales[["Item_Fat_Content"]]).toarray())
sales=sales.join(sales_OHE)
sales=sales.drop('Item_Fat_Content', axis=1)
sales=sales.rename(columns={0: "Fat_Content_low_Fat", 1:"Fat_Content_None",2:"Fat_Content_Regular"})

In [15]:
sales_OHE2=pd.DataFrame(OneHotEncoder().fit_transform(sales[["Item_Type"]]).toarray())
sales=sales.join(sales_OHE2)
sales=sales.drop('Item_Type', axis=1)
sales=sales.rename(columns={0:"Type_Baking_Goods", 1:"Type_Breads",2:"Type_Breakfast",3:"Type_Canned",4:"Type_Dairy",5:"Type_Frozen_Foods",6:"Type_Fruits_and_Vegetables",7:"Type_Hard_Drinks",8:"Type_Health_and_Hygiene",9:"Type_Household",10:"Type_Meat",11:"Type_Other",12:"Type_Seafood",13:"Type_Snack_Food",14:"Type_Soft_Drink",15:"Type_Starchy_Foods",})

In [16]:
sales_OHE3=pd.DataFrame(OneHotEncoder().fit_transform(sales[["Outlet_Size"]]).toarray())
sales=sales.join(sales_OHE3)
sales=sales.drop('Outlet_Size', axis=1)
sales=sales.rename(columns={0: "Size_NO_Size", 1:"Size_High",2:"Size_Medium",3:"Size_Small"})

In [17]:
sales_OHE4=pd.DataFrame(OneHotEncoder().fit_transform(sales[["Outlet_Location_Type"]]).toarray())
sales=sales.join(sales_OHE4)
sales=sales.drop('Outlet_Location_Type', axis=1)
sales=sales.rename(columns={0: "Location_Type_Tier_1", 1:"Location_Type_Tier_2",2:"Location_Type_Tier_3"})

In [18]:
sales_OHE5=pd.DataFrame(OneHotEncoder().fit_transform(sales[["Outlet_Type"]]).toarray())
sales=sales.join(sales_OHE5)
sales=sales.drop('Outlet_Type', axis=1)
sales=sales.rename(columns={0: "Type_Grocery_Store", 1:"Type_Supermarket_Type1",2:"Type_Supermarket_Type2",3:"Type_Supermarket_Type3"})

In [19]:
sales['Item_Outlet_Sales'] = pd.to_numeric(sales['Item_Outlet_Sales'], errors='coerce')
for x, y in zip(sales.Item_Outlet_Sales, sales.index): 
        rep1 = math.isnan(sales.Item_Outlet_Sales[y]) 
        if rep1 == True : 
            sales['Item_Outlet_Sales'][y] = sales['Item_Outlet_Sales'].mean()
#print(sales.Item_Outlet_Sales)
#sales.Item_Outlet_Sales.describe()

In [ ]:
randomforest=RandomForestRegressor(n_estimators=100,min_samples_leaf=1) 
y=sales['Item_Outlet_Sales']
x=sales.loc[:,sales.columns !='Item_Outlet_Sales']
#print(x)
randomforest.fit(x,y)
score=cross_val_score(randomforest,x,y,cv=70)
moyenne=score.mean()
y_predit=randomforest.predict(x) 
print(moyenne)